In [6]:
import pickle
import pystan
import numpy as np

In [7]:
# Generate artificial samples from Neg Bin.
artificial_samples = np.random.negative_binomial(10, 0.2, 1000)
mu = artificial_samples.mean()
phi = (mu ** 2) / (artificial_samples.var() - mu)
mu, phi

(40.49, 10.755653074707121)

In [ ]:
model = model_even_simpler_hier_negbin = pystan.StanModel(model_code='''
data {
    int<lower=0> N; // Number of data points.
    int y[N]; // data points.
    real prior_mu;
    real prior_phi;
}
parameters {
    real<lower=0> phi; // Prevent too low phi, which yields enormous variance in NegBin.
    real<lower=0> mu;
}
model {
    mu ~ normal(prior_mu, 10);
    phi ~ normal(prior_phi, 10);    
    y ~ neg_binomial_2(mu, phi);
}
''')

In [ ]:
with open('model_artificial_negbin.pkl', 'wb') as f:
    pickle.dump(model, f)

In [8]:
model = pickle.load(open('model_artificial_negbin.pkl', 'rb'))

In [11]:
fit = model.sampling(
    seed=1234, 
    data={'N': 1000, 'y': artificial_samples, 'prior_mu': mu, 'prior_phi': phi},
    iter=400, chains=1)


/home/inonpe/jupyter/dtu_bda_project/env/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [12]:
fit

Inference for Stan model: anon_model_4cf8d16a474380cc6c3df264b4082b69.
1 chains, each with iter=400; warmup=200; thin=1; 
post-warmup draws per chain=200, total post-warmup draws=200.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
phi   10.65    0.06   0.51   9.81  10.29  10.61  10.99  11.81     64    1.0
mu    40.46    0.03   0.44  39.65   40.2  40.45  40.78  41.33    158   1.01
lp__  1.1e5    0.07   0.82  1.1e5  1.1e5  1.1e5  1.1e5  1.1e5    128   1.01

Samples were drawn using NUTS at Thu Apr 12 10:31:22 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).

Conclusion: we use mu and phi correctly in neg_binomial_2.